In [6]:
import numpy as np
import pandas as pd

In [26]:
df = pd.read_csv("../../data/duty_textClean.csv")
df.head(5)

,Unnamed: 0,shixin_id,duty
0,1,2905,被告上海纺友贸易有限公司应于本判决生效后十日内支付原告江苏盛虹面料有限公司货款人民币 元如...
1,6,3650,一被告上海栩瑞电气设备有限公司确认尚欠原告上海宝临超亚电器有限公司货款 元 原告上海宝临超亚...
2,16,11617,原告上海玖峰数码科技有限公司于本判决生效之日起十日内支付被告张宁顺违法解除劳动合同赔偿金 元...
3,21,15642,被执行人于年月前向申请执行人支付工资人民币万元 被执行人于年月前向申请执行人支付工资人民币万...
4,22,16229,一原告上海顶天钢铁发展有限公司与被告内蒙古兴托物资有限公司于年月日签订的 内蒙古托县至兴和煤...


In [27]:
df.shape

(17255, 3)

In [28]:
df = df.sample(2000)

In [29]:
import jieba

In [31]:
# Need to be length greater than 10;

def length_bigger_than_10(text):
    if len(text)>=10:
        return True
    else:
        return False


#新建length列
df['length'] = df.agg({'duty': length_bigger_than_10})
#根据length列来保留长度大于10的文本，并将其放入新的dataframe中
df = df[df['length']==True]

In [33]:
import nltk
import jieba




In [51]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leijunjie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [55]:
#停止词
from nltk.corpus import stopwords
stopwords = stopwords.words("chinese")



def clean_text(text):
    wordlist = jieba.lcut(text) 
    #去除停用词和长度小于2的词语
    wordlist = [w for w in wordlist if w not in stopwords and len(w)>2]
    #将中文数据组织成类似西方于洋那样，词语之间以空格间隔
    document =  " ".join(wordlist)
    return document


#使用dataframe.agg对content列实行clean_text方法
df['duty'] = df.agg({'duty': clean_text})
corpus = df['duty']
corpus.head()

16270                                             
11601                                     陈显文 劳动报酬
14481                              卓苏榜 卓苏榜 张铁柱 张铁柱
664                                               
7607     肖芳文 金属材料 中国工商银行 高建明 肖芳文 金属材料 肖芳文 金属材料 高建明
Name: duty, dtype: object

In [11]:
## jieba 分词包

def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [12]:
df["duty_cutted"] = df.duty.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/ys/98s0q77d0bl4mpqsp4lf7c740000gn/T/jieba.cache
Loading model cost 0.769 seconds.
Prefix dict has been built successfully.


In [13]:
df.duty_cutted.head()

18924    判决 被告 偿还 借款 本金 人民币 3500000 元 ， 支付 截止 2013 年 5 ...
10166    要求 被执行人 胡文周 归还 借款 本金 5 , 500 , 000 元 并 支付 2011...
16713    一 、 被告 上海 景悅 企业 发展 （ 集团 ） 有限公司 、 邱 福昌 确认 欠付 原告...
4744     执行 被申请人 名下 车辆 拍卖 或 变卖 款 由 申请人 在 其 担保 范围 内 优先 受...
18094    一 、 被告 上海 环震 包装 制品 有限公司 于本 判决 生效 之日起 十日 内 赔偿 原...
Name: duty_cutted, dtype: object

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [57]:
stpwrdpath = "../../data/stopwords.txt"
stpwrd_dic = open(stpwrdpath, 'rb')
stpwrd_content = stpwrd_dic.read().decode('utf-8')#将停用词表转换为list
stpwrdlst = stpwrd_content.splitlines()
stpwrd_dic.close()

In [58]:
n_features = 1000

In [59]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                #stop_words='english',
                                stop_words=stpwrdlst,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.duty_cutted)

AttributeError: 'DataFrame' object has no attribute 'duty_cutted'

In [60]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                #stop_words='english',
                                stop_words=stpwrdlst,
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(corpus)

In [61]:
from sklearn.decomposition import LatentDirichletAllocation

In [65]:
lda = LatentDirichletAllocation(n_components = 10,max_iter=50,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0)

In [66]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50,
                          max_doc_update_iter=100, max_iter=50,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [67]:
import pyLDAvis 
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.016016  0.189466       1        1  13.604753
5     -0.177683  0.029101       2        1  13.285433
3      0.295042 -0.063205       3        1  11.093130
6      0.144607  0.135363       4        1  10.956377
1     -0.216991  0.017644       5        1  10.314835
0     -0.021436  0.089067       6        1  10.272496
8     -0.075347 -0.237122       7        1   8.880012
2     -0.072130  0.194712       8        1   7.921254
4     -0.081624 -0.289775       9        1   7.254717
9      0.221578 -0.065250      10        1   6.416993, topic_info=    Term        Freq       Total Category  logprob  loglift
42   房地产  178.000000  178.000000  Default  30.0000  30.0000
0    一次性   89.000000   89.000000  Default  29.0000  29.0000
95  连带保证   66.000000   66.000000  Default  28.0000  28.0000
12   保证金   63.000000   63.000000  Default  27.0000  27.0000
99  金属材料   83.000000   83.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
93   赔偿金    0.105054   39.971248  Topic10  -7.7921  -3.1952
18  包装材料    0.105054   26.185829  Topic10  -7.7921  -2.7723
8   人民法院    0.105053   28.768129  Topic10  -7.7921  -2.8663
10   使用费    0.105053   58.497330  Topic10  -7.7921  -3.5760
9    代理费    0.105053   43.458893  Topic10  -7.7921  -3.2789

[463 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0        10  0.984696   一次性
1         2  0.695326  万分之三
1         5  0.146385  万分之三
1         7  0.146385  万分之三
2         1  0.953535   三十日
...     ...       ...   ...
104      10  0.951947   闵行区
105       2  0.951761   陈佳信
106       4  0.978403   陈芳任
107       1  0.983560   青浦区
108       2  0.951761   魏祖斌

[148 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 6, 4, 7, 2, 1, 9, 3, 5, 10])

In [68]:
p = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

In [69]:
pyLDAvis.save_html(p, 'lda.html')